In [ ]:
import numpy as np
import xarray as xr
import sys
import xgcm
from joblib import Parallel, delayed
import eddytools as et

# year=1951
# mask=np.load('/work/ab0995/a270166/SO_analysis/high-lat-analysis/ice33_region_1951-1955_mask.npy')

depth='-27.5m' #'-2.5m' '-195.0m' '-97.5m'

def define_grid(gridtype='reg',bounds=[-180,180,-80,90],dx=1,dy=1,periodic=True):
    left,right,bottom,top=bounds
    #variables we want to keep
    global lons_c, lats_c, lons_gl, lats_gl, lons_gr, lats_gr, lats_g, \
    vpointslonl, vpointslatl, vpointslonr, vpointslatr, upointslonl, upointslatl, upointslonr, \
    upointslatr, cpointslon, cpointslat, fpointslonl, fpointslatl, fpointslonr, fpointslatr, \
    dxF, dyF, dxC, dyC, dxG, dyG, dxV, dyU 
    
    if gridtype=='reg':
        # the x center and f points on either side
        lons_c = np.arange(left+(0.5*dx), right, dx)
        lons_gl = np.arange(left, right, dx)
        lons_gr = lons_gl+(dx)
        
        lats_c = np.arange(bottom+(0.5*dy), top, dy)
        lats_gl = np.arange(bottom, top, dy)
        lats_gr = lats_gl+(dy)
        
    elif gridtype=='iso':
        
        # the x center and f points on either side
        lons_c = np.arange(left+(0.5*dx), right, dx)
        lons_gl = np.arange(left, right, dx)
        lons_gr = lons_gl+(dx)
        
        #the y 
        lats_g=[bottom]

        #we need one extra center lat point in order to differentiate the correct number of v points
        while lats_g[-1] < top+(dy*(np.cos(np.radians(top)))):
            lats_g.append(lats_g[-1]+dy*(np.cos(np.radians(lats_g[-1]))))

        lats_gl=lats_g[:-1]
        lats_gr=lats_g[1:]
        lats_c=(np.asarray(lats_gl)+np.asarray(lats_gr))/2

        lats_g,lats_gl,lats_gr=lats_g[:-1],lats_gl[:-1],lats_gr[:-1]
        
    else:
        print('grid type not supported')
        pass
        
    #latlon points in 2d
    vpointslonl, vpointslatl = np.meshgrid(lons_c,lats_gl)
    vpointslonr, vpointslatr = np.meshgrid(lons_c,lats_gr)

    upointslonl, upointslatl = np.meshgrid(lons_gl,lats_c)
    upointslonr, upointslatr = np.meshgrid(lons_gr,lats_c)

    cpointslon, cpointslat = np.meshgrid(lons_c,lats_c)

    fpointslonl, fpointslatl = np.meshgrid(lons_gl,lats_gl)
    fpointslonr, fpointslatr = np.meshgrid(lons_gr,lats_gr)
    
    #distances
    dxC = cpointslon[:-1,1:] - cpointslon[:-1,:-1]
    #we use the left lat point because we lose the top row 
    dxV = vpointslonl[:,1:] - vpointslonl[:,:-1]
    #for dxC and dxV we need to manually add the periodic distance
    if periodic:
        endcolumn=(cpointslon[:-1,0]-cpointslon[:-1,-1])+[360]
        dxC = np.append(dxC, endcolumn.reshape((endcolumn.shape[0],1)),axis=1)
        endcolumn=(vpointslonl[:,0]-vpointslonl[:,-1])+[360]
        dxV = np.append(dxV, endcolumn.reshape((endcolumn.shape[0],1)),axis=1)

    #for dyC and dyU, we lose the top row that we added above
    #*** to do: change to lose the bottom row, so that this works better in the northern hemisphere
    #the bottom row can be on land over antarctica
    dyC = cpointslat[1:,:] - cpointslat[:-1,:]
    dyU = upointslatl[1:,:] - upointslatl[:-1,:]

    #dxG and dyG
    dxG = fpointslonr-fpointslonl
    dyG = fpointslatr-fpointslatl

    #dxF and dyF. for dxF we have to manually remove the top layer
    dxF = upointslonr - upointslonl
    dxF = dxF[:-1,:]
    dyF = vpointslatr - vpointslatl
    
    lats_c=lats_c[:-1]
    cpointslat=cpointslat[:-1,:]
    upointslonl=upointslonl[:-1,:]
    upointslatl=upointslatl[:-1,:]
    upointslonr=upointslonr[:-1,:]
    upointslatr=upointslatr[:-1,:]

define_grid('iso',[-180,180,-80,-40],dx=0.05,dy=0.05, periodic=True)





In [ ]:
def save(day,year):
    if day ==365 and year%4 !=0:
        return np.nan
    else:
        uvdata=xr.open_dataset('/PATH/TO/DATA/uv_uvgrid_'+depth+'_10-100kmbp_'+str(year)+'_'+str(day).zfill(4)+'.nc')

        #define the metrics
        metrics = {('X'): ['dxC', 'dxG', 'dxF', 'dxV'], # X distances
                ('Y'): ['dyC', 'dyG', 'dyF', 'dyU']}
        # define the final grid
        grid = xgcm.Grid(uvdata, 
                         coords={'X': {'center': 'XC', 'left': 'XG'},
                                 'Y': {'center': 'YC', 'left': 'YG'},
                                 'Z': {'center': 'Z' , 'right': 'Zp1'},
                                 }, 
                         periodic={'X':True,'Y':False,'Z':False}, 
                         autoparse_metadata=False,
                         metrics=metrics)

        data_OW=et.okuboweiss.calc(uvdata, grid, 'UVEL', 'VVEL')
        del uvdata

        mean_OW_spatial_std = np.nanstd(data_OW.OW[0,0].sel({'YG':slice(-80,-60)}).values)
        del data_OW

        return mean_OW_spatial_std

In [ ]:
ow_spatial=Parallel(n_jobs=-1,batch_size=1,verbose=10)(delayed(save)(day,year) for day in np.arange(0,366,1) for year in [1951,1952,1953,1954,1955,1956])

ows=np.nanmean(np.vstack(ow_spatial))

np.save('/PATH/TO/OUTPUT/OW_std_'+depth+'_impfil10-100_1951-1956.npy',ows)